Author: S08920611 & S08920030 from Sustainability Science and Management Dept., Tunghai University
Please get data from gisaid-variants-statistics at gisaid.org !

In [146]:
import os,tarfile,math
import pandas as pd
import numpy as np
import xml.dom.minidom
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense,Dropout
from nltk import word_tokenize,pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def Extract_file(file_path):  #Extract downloaded file from GISAID
    print("---Extract---")
    if os.path.isfile(file_path):
        tar=tarfile.open(file_path,'r:xz')
        tar.extractall()
        tar.close()
        print("Extract finish.")
    elif file_path=='skip':
        print("Extract part skipped.")
    else:
        print("Error: file does not exist, please check the path.")
def Preprocess():  #Data preprocess
    print("---Data preprocess---")
    df = pd.read_json("gisaid_variants_statistics.json",encoding="utf-8", orient='records')
    global db
    db=pd.DataFrame(index=range(df.shape[0]),columns=['Time', 'Species', 'AA_Substitution', 'Lineage'])
    dataset=[]
    db=pd.DataFrame(columns=['Time', 'Species', 'AA_Substitution', 'Lineage'])
    df=df.rename_axis('time').reset_index()
    global spikeset  #Collect all spike variants
    spikeset=set()
    for i in range(df.index[-1]):
        country=str(df.iloc[i]["stats"])[2:str(df.iloc[i]["stats"]).find(":")-1]
        df.iloc[i]["stats"][country]
        db.loc[len(db.index)]=(int(str(df.iloc[i]["time"])[:4])-2010)*365+int(str(df.iloc[i]["time"])[5:7])*30+int(str(df.iloc[i]["time"])[8:10]),df.iloc[i]["stats"][country]['submissions_per_variant'][0]['value'],df.iloc[i]["stats"][country]['submissions_per_aa_substitution'],df.iloc[i]["stats"][country]['submissions_per_lineage']
        if len(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'])>1:
            db.loc[i,'AA_Substitution']=['']
            for j in range(int(len(df.iloc[i]["stats"][country]['submissions_per_aa_substitution']))):
                db.iloc[i]['AA_Substitution'].append(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'][j]['value'])
                spikeset.add(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'][j]['value'])
            db.loc[i,'AA_Substitution'].remove('')
        elif len(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'])==1:
            db.loc[i,'AA_Substitution']=['']
            db.iloc[i]['AA_Substitution'].append(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'][0]['value'])
            db.loc[i,'AA_Substitution'].remove('')
            spikeset.add(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'][0]['value'])
        if len(df.iloc[i]["stats"][country]['submissions_per_lineage'])>1:
            db.loc[i,'Lineage']=['']
            for j in range(int(len(df.iloc[i]["stats"][country]['submissions_per_lineage']))):
                db.iloc[i]['Lineage'].append(df.iloc[i]["stats"][country]['submissions_per_lineage'][j]['value'])
            db.loc[i,'Lineage'].remove('')
        elif len(df.iloc[i]["stats"][country]['submissions_per_lineage'])==1:
            db.loc[i,'Lineage']=['']
            db.iloc[i]['Lineage'].append(df.iloc[i]["stats"][country]['submissions_per_lineage'][0]['value'])
            db.loc[i,'Lineage'].remove('')
    print("Data preprocess done.")
def Model_Training():
    print("---Model training---")
    y = list(spikeset)
    le = LabelEncoder()  #Encode spikes into array instead of spike names
    le.fit(y)
    ylist=[]
    for i in range(db.values.T[2].shape[0]):
        ylist.append(list(le.transform(db.values.T[2][i])))
    ylistarr=np.zeros((len(ylist),len(list(spikeset))))
    for i in range (len(ylist)):
        for j in range(len(list(spikeset))):
            if j in ylist[i]:
                ylistarr[i,j]=1
    data_x = db.values.T[0].astype(np.float32).reshape(-1,1)
    data_y = ylistarr.astype(np.float32)
    xtrain, xtest, ytrain, ytest = train_test_split(data_x,data_y,test_size=0.25,random_state=1)
    global model
    model = Sequential()
    model.add(Dense(units=1000,kernel_initializer='normal',activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=10000,kernel_initializer='normal',activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=data_y.shape[1],kernel_initializer='normal',activation='softmax'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    train_history=model.fit(x=xtrain,y=ytrain,validation_split=0.2,epochs=50,batch_size=200,verbose=0)
    xpred=model.predict(xtest)
    accuracy=0
    for i in range(ytest.shape[0]):  #Rewrite accuracy function
        for j in range(ytest.shape[1]):
            if ytest[i][j]==xpred[i][j]:
                accuracy+=1
    accuracy=accuracy/ytest.shape[0]/ytest.shape[1] 
    print('accuracy',accuracy)
    print("Model training done.")
def Predict_Spike():
    print("---Predict spike---")
    timestr=""
    if len(timestr)!=10:  #Enter predicting date 
        timestr=input("Enter the time that you want to predict(Ex: 2023.01.01): ")
        if len(timestr)!=10:
            print("Invalid input, please check and enter again!")
    time=float((int(timestr[:4])-2010)*365+int(timestr[5:7])*30+int(timestr[8:10]))
    pred=model.predict(np.array(time).reshape(-1,1))[0]
    global spike
    spike=[]
    for i in range(pred.shape[0]):
        if pred[i]==0:
            predlog10=-10
        else:
            predlog10=(math.log10(abs(pred[i])))*-1
        if predlog10>-10:
            spike.append(list(spikeset)[i])     #Simple predict function(Could replaced by a more convincing function)
    print("The new virus might contain:",str(spike)[1:len(str(spike))-1])
    print("Predict spike done.")
def Predict_Effect():
    print("---Predict effect---")
    txt=open('Mutations and Evolution of the SARS-CoV-2 Spike Protein.txt','r',encoding="utf-8")
    document=txt.read()
    global sentence
    sentence=[]
    for i in range(len(spike)):
        if document.find(str(spike[i])[str(spike[i]).find("_")+1:10])!=-1:
            sentence.append(document[document[:document.find(str(spike[i])[str(spike[i]).find("_")+1:10])].rfind(".")+2:document[document.find(str(spike[i])[str(spike[i]).find("_")+1:]):].find(".")+document.find(str(spike[i])[str(spike[i]).find("_")+1:10])])
    cutwords5=[]
    for i in range (len(sentence)):
        cutwords1 = word_tokenize(sentence[i])
        interpunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
        cutwords2 = [word for word in cutwords1 if word not in interpunctuations]
        stops = set(stopwords.words("english"))
        cutwords3 = [word for word in cutwords2 if word not in stops]
        cutwords4 = []
        for cutword in cutwords3:
            cutwords4.append(PorterStemmer().stem(cutword))
        cutwords5.append(cutwords4)
    vocab=[['neutral','diminish','evad','impair','immune-evas','resist','lost-act','escap'],['mouse-adapt','mous'],['infect','bind','viral-load','receptor-bind','receptor','transmiss','spread'],['destabil']]
    #List1:Immune escape capability; 2:Zoonosis 3.Rapidly dissemination ability 4.Mutability
    Immune_escape_capability=Zoonosis=Rapidly_dissemination_ability=Mutability=count=0
    for i in range(len(cutwords5)):
        for j in range(len(cutwords5[i])):
            if cutwords5[i][j] in vocab[0]:
                Immune_escape_capability+=1
            if cutwords5[i][j] in vocab[1]:
                Zoonosis+=1
            if cutwords5[i][j] in vocab[2]:
                Rapidly_dissemination_ability+=1
            if cutwords5[i][j] in vocab[3]:
                Mutability+=1
    print("The possible property of the new virus is(are): ",end="")
    count=0
    if Immune_escape_capability>0:
        count+=1
        print(str(Immune_escape_capability)+" times of immune escape capability, ",end="")
    if Zoonosis>0:
        count+=1
        print(str(Zoonosis)+" times of zoonosis, ",end="")
    if Rapidly_dissemination_ability>0:
        count+=1
        print(str(Rapidly_dissemination_ability)+" times of rapidly dissemination ability, ",end="")
    if Mutability>0:
        count+=1
        print(str(Mutability)+" times of mutability, ",end="")
    if count==0:
        print("No special property.")
    else:
        print("than the origin COVID-19.")
    print("Predict effect done.")

In [147]:
def main():
    Extract_file(input("Put the data file in the same path, and enter the name of the file(Ex:gisaid_variants_statistics_2022_12_27_1557.tar.xz)(if already extracted, enter ‘skip’): "))
    Preprocess()
    Model_Training()
    Predict_Spike()
    Predict_Effect()
main()

---Extract---
Extract part skipped.
---Data preprocess---
Data preprocess done.
---Model training---
2/2 [==============================] - 0s 5ms/step
accuracy 0.5395121951219513
Model training done.
---Predict spike---
1/1 [==============================] - 0s 22ms/step
The new virus might contain: 'Spike_N440K', 'Spike_R408S', 'Spike_N460K', 'Spike_L368I', 'Spike_T478K', 'Spike_R346T', 'Spike_K356T', 'Spike_S494P', 'Spike_G446S', 'Spike_T376A', 'Spike_P681H', 'Spike_F486S', 'Spike_N501Y', 'Spike_H69del', 'Spike_G339D', 'Spike_S375F', 'Spike_D614G'
Predict spike done.
---Predict effect---
The possible property of the new virus is(are): 7 times of immune escape capability, 8 times of rapidly dissemination ability, than the origin COVID-19.
Predict effect done.


In [44]:
import pandas as pd
df = pd.read_json("gisaid_variants_statistics.json",encoding="utf-8", orient='records')
dataset=[]
db=pd.DataFrame(columns=['Time', 'Species', 'AA_Substitution', 'Lineage'])
df=df.rename_axis('time').reset_index()
spikeset=set()
for i in range(df.index[-1]):
    country=str(df.iloc[i]["stats"])[2:str(df.iloc[i]["stats"]).find(":")-1]
    df.iloc[i]["stats"][country]
    db.loc[len(db.index)]=(int(str(df.iloc[i]["time"])[:4])-2010)*365+int(str(df.iloc[i]["time"])[5:7])*30+int(str(df.iloc[i]["time"])[8:10]),df.iloc[i]["stats"][country]['submissions_per_variant'][0]['value'],df.iloc[i]["stats"][country]['submissions_per_aa_substitution'],df.iloc[i]["stats"][country]['submissions_per_lineage']
    if len(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'])>1:
        db.loc[i,'AA_Substitution']=['']
        for j in range(int(len(df.iloc[i]["stats"][country]['submissions_per_aa_substitution']))):
            db.iloc[i]['AA_Substitution'].append(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'][j]['value'])
            spikeset.add(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'][j]['value'])
        db.loc[i,'AA_Substitution'].remove('')
    elif len(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'])==1:
        db.loc[i,'AA_Substitution']=['']
        db.iloc[i]['AA_Substitution'].append(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'][0]['value'])
        spikeset.add(df.iloc[i]["stats"][country]['submissions_per_aa_substitution'][0]['value'])
        db.loc[i,'AA_Substitution'].remove('')
    if len(df.iloc[i]["stats"][country]['submissions_per_lineage'])>1:
        db.loc[i,'Lineage']=['']
        for j in range(int(len(df.iloc[i]["stats"][country]['submissions_per_lineage']))):
            db.iloc[i]['Lineage'].append(df.iloc[i]["stats"][country]['submissions_per_lineage'][j]['value'])
        db.loc[i,'Lineage'].remove('')
    elif len(df.iloc[i]["stats"][country]['submissions_per_lineage'])==1:
        db.loc[i,'Lineage']=['']
        db.iloc[i]['Lineage'].append(df.iloc[i]["stats"][country]['submissions_per_lineage'][0]['value'])
        db.loc[i,'Lineage'].remove('')


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np
y = list(spikeset)
le = LabelEncoder()
le.fit(y)
ylist=[]
for i in range(db.values.T[2].shape[0]):
    ylist.append(list(le.transform(db.values.T[2][i])))
ylistarr=np.zeros((len(ylist),len(list(spikeset))))
for i in range (len(ylist)):
    for j in range(len(list(spikeset))):
        if j in ylist[i]:
            ylistarr[i,j]=1
data_x = db.values.T[0].astype(np.float32).reshape(-1,1)
data_y = ylistarr.astype(np.float32)

In [52]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(data_x,data_y,test_size=0.2)
model2 = Sequential()
model2.add(Dense(units=1000,kernel_initializer='normal',activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(units=1000,kernel_initializer='normal',activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(units=data_y.shape[1],kernel_initializer='normal',activation='softmax'))
model2.compile(loss='binary_crossentropy',optimizer='adam')

train_history=model2.fit(x=xtrain,y=ytrain,validation_split=0.2,epochs=20,batch_size=200,verbose=0)
xpred=model.predict(xtest)
accuracy=0
for i in range(ytest.shape[0]):  #Rewrite accuracy function
    for j in range(ytest.shape[1]):
        if ytest[i][j]==xpred[i][j]:
            accuracy+=1
accuracy=accuracy/ytest.shape[0]/ytest.shape[1] 
print('accuracy',accuracy)
print(model2.summary())

2/2 [==============================] - 0s 4ms/step
accuracy 0.6842424242424242
Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_94 (Dense)            (None, 1000)              2000      
                                                                 
 dropout_63 (Dropout)        (None, 1000)              0         
                                                                 
 dense_95 (Dense)            (None, 1000)              1001000   
                                                                 
 dropout_64 (Dropout)        (None, 1000)              0         
                                                                 
 dense_96 (Dense)            (None, 50)                50050     
                                                                 
Total params: 1,053,050
Trainable params: 1,053,050
Non-trainable params: 0
______________________________

In [66]:
import math
timestr=""
if len(timestr)!=10:
    timestr=input("Enter the time that you want to predict(Ex: 2023.01.01): ")
    if len(timestr)!=10:
        print("Invalid input, please check and enter again!")
time=float((int(timestr[:4])-2010)*365+int(timestr[5:7])*30+int(timestr[8:10]))
pred=model2.predict(np.array(time).reshape(-1,1))[0]
spike=[]
for i in range(pred.shape[0]):
    if math.log10(pred[i])>-10:
        spike.append(list(spikeset)[i])
print(spike)

1/1 [==============================] - 0s 17ms/step
['Spike_K444N', 'Spike_L368I', 'Spike_T478K', 'Spike_R346T', 'Spike_S373P', 'Spike_T376A', 'Spike_P681H', 'Spike_G339D', 'Spike_S375F', 'Spike_D614G', 'Spike_Q498R']


In [68]:
txt=open('Mutations and Evolution of the SARS-CoV-2 Spike Protein.txt','r',encoding="utf-8")
document=txt.read()


In [69]:
global sentence
sentence=[]
for i in range(len(spike)):
    if document.find(str(spike[i])[str(spike[i]).find("_")+1:10])!=-1:
        print(str(spike[i])[str(spike[i]).find("_")+1:10])
        sentence.append(document[document[:document.find(str(spike[i])[str(spike[i]).find("_")+1:10])].rfind(".")+2:document[document.find(str(spike[i])[str(spike[i]).find("_")+1:]):].find(".")+document.find(str(spike[i])[str(spike[i]).find("_")+1:10])])

T478
P681
G339
D614
Q498


In [87]:
from nltk import word_tokenize,pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
txt=open('Mutations and Evolution of the SARS-CoV-2 Spike Protein.txt','r',encoding="utf-8")
document=txt.read()
global sentence
sentence=[]
for i in range(len(spike)):
    if document.find(str(spike[i])[str(spike[i]).find("_")+1:10])!=-1:
        print(str(spike[i])[str(spike[i]).find("_")+1:10])
        sentence.append(document[document[:document.find(str(spike[i])[str(spike[i]).find("_")+1:10])].rfind(".")+2:document[document.find(str(spike[i])[str(spike[i]).find("_")+1:]):].find(".")+document.find(str(spike[i])[str(spike[i]).find("_")+1:10])])
cutwords5=[]
for i in range (len(sentence)):
    cutwords1 = word_tokenize(sentence[i])
    interpunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
    cutwords2 = [word for word in cutwords1 if word not in interpunctuations]
    stops = set(stopwords.words("english"))
    cutwords3 = [word for word in cutwords2 if word not in stops]
    cutwords4 = []
    for cutword in cutwords3:
        cutwords4.append(PorterStemmer().stem(cutword))
    cutwords5.append(cutwords4)
vocab=[['neutral','diminish','evad','impair','immune-evas','resist','lost-act','escap'],['mouse-adapt','mous'],['infect','bind','viral-load','receptor-bind','receptor','transmiss','spread'],['destabil']]
#List1:Immune escape capability; 2:Zoonosis 3.Rapidly dissemination ability 4.Mutability
Immune_escape_capability=Zoonosis=Rapidly_dissemination_ability=Mutability=count=0
for i in range(len(cutwords5)):
    for j in range(len(cutwords5[i])):
        if cutwords5[i][j] in vocab[0]:
            Immune_escape_capability+=1
        if cutwords5[i][j] in vocab[1]:
            Zoonosis+=1
        if cutwords5[i][j] in vocab[2]:
            Rapidly_dissemination_ability+=1
        if cutwords5[i][j] in vocab[3]:
            Mutability+=1
print("The possible property of the new virus is(are): ",end="")
if Immune_escape_capability>0:
    print(str(Immune_escape_capability)+" times of immune escape capability, ",end="")
if Zoonosis>0:
    print(str(Zoonosis)+" times of zoonosis, ",end="")
if Rapidly_dissemination_ability>0:
    print(str(Rapidly_dissemination_ability)+" times of rapidly dissemination ability, ",end="")
if Mutability>0:
    print(str(Mutability)+" times of mutability, ",end="")

The possible property of the new virus is(are): Immune escape capability, Rapidly dissemination ability, 

In [127]:
print(spikeset)

{'Spike_N440K', 'Spike_D796Y', 'Spike_D405N', 'Spike_R408S', 'Spike_K444N', 'Spike_N460K', 'Spike_L368I', 'Spike_G142D', 'Spike_T478K', 'Spike_Y145del', 'Spike_G339H', 'Spike_F486P', 'N_E136D', 'Spike_S255F', 'Spike_G257S', 'Spike_D253G', 'Spike_W152R', 'Spike_F490S', 'Spike_K147E', 'Spike_R346T', 'Spike_K356T', 'Spike_S371F', 'Spike_S373P', 'Spike_S494P', 'Spike_V445P', 'Spike_G446S', 'Spike_L452R', 'Spike_E484A', 'Spike_V70del', 'N_P13L', 'Spike_T376A', 'Spike_P251L', 'Spike_P681H', 'Spike_F486S', 'Spike_H655Y', 'Spike_S477N', 'Spike_Y144del', 'Spike_N501Y', 'NS7a_P84L', 'Spike_H69del', 'Spike_G339D', 'Spike_K417N', 'Spike_K444T', 'Spike_S375F', 'Spike_Y505H', 'Spike_D614G', 'Spike_Q498R', 'E_T11A', 'Spike_H146Q', 'Spike_F486V'}


In [145]:
PorterStemmer().stem("mouse")

'mous'